<p align="center"><img width="50%" src="https://aimodelsharecontent.s3.amazonaws.com/aimodshare_banner.jpg" /></p>

# Climate Change Satellite Image Classification Competition Model Submission Guide - PyTorch

---
**About the Original Data:**<br>
*Data and Description accessed from [Tensorflow](https://www.tensorflow.org/datasets/catalog/bigearthnet)* <br>
The BigEarthNet is a new large-scale Sentinel-2 benchmark archive, consisting of 590,326 Sentinel-2 image patches. The image patch size on the ground is 1.2 x 1.2 km with variable image size depending on the channel resolution. This is a multi-label dataset with 43 imbalanced labels, which has been simplified to single labels with 3 categories for the purposes of this competition.

To construct the BigEarthNet, 125 Sentinel-2 tiles acquired between June 2017 and May 2018 over the 10 countries (Austria, Belgium, Finland, Ireland, Kosovo, Lithuania, Luxembourg, Portugal, Serbia, Switzerland) of Europe were initially selected. All the tiles were atmospherically corrected by the Sentinel-2 Level 2A product generation and formatting tool (sen2cor). Then, they were divided into 590,326 non-overlapping image patches. Each image patch was annotated by the multiple land-cover classes (i.e., multi-labels) that were provided from the CORINE Land Cover database of the year 2018 (CLC 2018).

Bands and pixel resolution in meters:

    B01: Coastal aerosol; 60m
    B02: Blue; 10m
    B03: Green; 10m
    B04: Red; 10m
    B05: Vegetation red edge; 20m
    B06: Vegetation red edge; 20m
    B07: Vegetation red edge; 20m
    B08: NIR; 10m
    B09: Water vapor; 60m
    B11: SWIR; 20m
    B12: SWIR; 20m
    B8A: Narrow NIR; 20m

License: Community Data License Agreement - Permissive, Version 1.0."

**Competition Data Specifics:**<br>
For the purpose of this competition, the original BigEarthNet dataset has been simplified to 20,000 images (15,000 training images and 5,000 test images) with 3 categories: "forest", "nonforest", and "snow_shadow_cloud", which contains images of snow and clouds. <br>
Each "image" is a folder with 12 satellite image layers, each of which pics up on different features. The example preprocessor uses just three layers: B02, B03, and B04, which contain the standard RGB layers used in ML models. However, you are free to use any combination of the satellite image layers. 

**Data Source:**<br>
Sumbul, G, Charfuelan, M, Demir, B and Markl, V. (2019). BigEarthNet: A Large-Scale Benchmark Archive For Remote Sensing Image Understanding. *Computing Research Repository (CoRR), abs/1902.06148.* https://www.tensorflow.org/datasets/catalog/bigearthnet




# Overview
---

Let's share our models to a centralized leaderboard, so that we can collaborate and learn from the model experimentation process...

**Instructions:**
1.   Get data in and set up X_train / X_test / y_train
2.   Preprocess data / Write and Save Preprocessor function
3. Fit model on preprocessed data and save preprocessor function and model 
4. Generate predictions from X_test data and submit model to competition
5. Repeat submission process to improve place on leaderboard



## 1. Load Data

In [1]:
#install aimodelshare library
! pip install aimodelshare-nightly

     |████████████████████████████████| 132 kB 24.9 MB/s 
     |████████████████████████████████| 274 kB 44.8 MB/s 
     |████████████████████████████████| 131 kB 42.1 MB/s 
     |████████████████████████████████| 5.2 MB 27.8 MB/s 
     |████████████████████████████████| 78 kB 3.5 MB/s 
     |████████████████████████████████| 300 kB 7.2 MB/s 
     |████████████████████████████████| 435 kB 12.4 MB/s 
     |████████████████████████████████| 127 kB 45.0 MB/s 
     |████████████████████████████████| 22.3 MB 9.0 MB/s 
     |████████████████████████████████| 178 kB 45.5 MB/s 
     |████████████████████████████████| 146 kB 1.6 MB/s 
     |████████████████████████████████| 96 kB 2.7 MB/s 
     |████████████████████████████████| 12.8 MB 39.8 MB/s 
     |████████████████████████████████| 7.7 MB 23.7 MB/s 
     |████████████████████████████████| 79 kB 6.7 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 87 kB 4.0 MB/s 
     |███████████████████

In [2]:
# Get competition data - May take a couple minutes due to size of data set
from aimodelshare import download_data
download_data('public.ecr.aws/y2e2a1d6/climate_competition_data-repository:latest') 


Data downloaded successfully.


In [3]:
# Unzip Data - May take a couple minutes due to size of data set
import zipfile
with zipfile.ZipFile('climate_competition_data/climate_competition_data.zip', 'r') as zip_ref:
    zip_ref.extractall('competition_data')

##2.   Preprocess data / Write and Save Preprocessor function


### **Write a Preprocessor Function**


> ###   Preprocessor functions are used to preprocess data into the precise data your model requires to generate predictions.  

*  *Preprocessor functions should always be named "preprocessor".*
*  *You can use any Python library in a preprocessor function, but all libraries should be imported inside your preprocessor function.*  
*  *For image prediction models users should minimally include function inputs for an image filepath and values to reshape the image height and width.*  


In [4]:
# Set up for data preprocessing
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import torchvision.transforms.functional

In [11]:
# Here is a pre-designed preprocessor, but you could also build your own to prepare the data differently

def preprocessor(imageband_directory):
        """
        This function preprocesses reads in images, resizes them to a fixed shape and
        min/max transforms them before converting feature values to float32 numeric values
        required by onnx files.
        
        params:
            imageband_directory
                path to folder with 13 satellite image bands
                      
        returns:
            X
                numpy array of preprocessed image data
                  
        """
           
        import PIL
        import os
        import numpy as np
        import tensorflow_datasets as tfds

        def _load_tif(data):
            """Loads TIF file and returns as float32 numpy array."""
            img=tfds.core.lazy_imports.PIL_Image.open(data)
            # img = img.resize((224,224),resample=PIL.Image.NEAREST)
            img = np.array(img.getdata()).reshape(img.size).astype(np.float32)
            return img

        image_list = []
        filelist1=os.listdir(imageband_directory)
        for fpath in filelist1:
          fullpath=imageband_directory+"/"+fpath
          if fullpath.endswith(('B02.tif','B03.tif','B04.tif')):
              imgarray=_load_tif(imageband_directory+"/"+fpath)
              # imgarray = cv2.resize(imgarray, (224,224), interpolation = cv2.INTER_AREA)
              image_list.append(imgarray)

        X = np.stack(image_list,axis=0)   # to get (3,height,width)
        # X = torchvision.transforms.functional.resize(X,size=[224,224])

        X = np.expand_dims(X, axis=0) # Expand dims to add "1" to object shape [1, h, w, channels] for keras model.
        X = np.array(X, dtype=np.float32) # Final shape for onnx runtime.
        X=X/18581 # min max transform to max value
        return X

In [12]:
# Create complete list of file names
forestfilenames=["competition_data/trainingdata/forest/"+x for x in os.listdir("competition_data/trainingdata/forest")]
nonforestfilenames=["competition_data/trainingdata/nonforest/"+x for x in os.listdir("competition_data/trainingdata/nonforest")]
otherfilenames=["competition_data/trainingdata/other/"+x for x in os.listdir("competition_data/trainingdata/other")]

filenames=forestfilenames+nonforestfilenames+otherfilenames

#preprocess rbg images into 120,120,3 numpy ndarray
preprocessed_image_data=[]
for i in filenames:
  try:
    preprocessed_image_data.append(preprocessor(i))
  except:
    pass  

In [13]:
# Set up y data
from itertools import repeat
forest=repeat("forest",5000)
nonforest=repeat("nonforest",5000)
other=repeat("snow_shadow_cloud",5000)
ylist=list(forest)+list(nonforest)+list(other)

In [14]:
# Shuffle X and y data
from sklearn.utils import shuffle
X, y = shuffle(preprocessed_image_data, ylist, random_state=0)

In [15]:
X =np.vstack(X) # convert X from list to array
# X_ = np.zeros([15000,3,224,224])
# for i in range(len(X)):
#   X_[i] = X[i]

In [10]:
X.shape

(15000, 3, 60, 60)

In [16]:
# get numerical representation of y labels
import pandas as pd
y_labels_num = pd.DataFrame(y)[0].map({'forest': 0, 'nonforest': 1, 'snow_shadow_cloud': 2}) 

y_labels_num = list(y_labels_num)

In [17]:
# Separate 20% of Data for validation
X_train = X[0:12000]
X_val = X[12001:15000]
y_train = y_labels_num[0:12000]
y_val = y_labels_num[12001:15000]

In [18]:
X_train.shape

(12000, 3, 120, 120)

In [19]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms, models
from torchvision.transforms import ToTensor

In [20]:
class ClimateDataset(torch.utils.data.Dataset):
  def __init__(self,dataset,labels):
    self.y = labels
    self.x = dataset
    print("ClimateDataset shape {}, {}".format(self.x.shape, self.y.shape))
  def __len__(self):
    return len(self.y)
  def __getitem__(self,idx):
    # self.x = preprocessor(filenames[i])
    # self.x = transform(self.x)
    # self.x = cv2.resize(self.x, (224,224), interpolation = cv2.INTER_AREA)
    # return self.x[idx],self.y[idx]
    return torch.squeeze(nn.functional.interpolate(self.x[idx].unsqueeze(0), (224,224)),0),self.y[idx]

##3. Fit model on preprocessed data and save preprocessor function and model 

### **Prepare Data** for Pytorch

In [21]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [22]:
# prepare datasets for pytorch dataloader
tensor_X_train = torch.Tensor(X_train)
tensor_y_train = torch.tensor(y_train, dtype=torch.long) 
# train_ds = TensorDataset(tensor_X_train, tensor_y_train)
train_ds = ClimateDataset(tensor_X_train, tensor_y_train) 

tensor_X_test = torch.Tensor(X_val) 
tensor_y_test = torch.tensor(y_val, dtype=torch.long)
# test_ds = TensorDataset(tensor_X_test,tensor_y_test)
test_ds = ClimateDataset(tensor_X_test, tensor_y_test)

ClimateDataset shape torch.Size([12000, 3, 120, 120]), torch.Size([12000])
ClimateDataset shape torch.Size([2999, 3, 120, 120]), torch.Size([2999])


In [23]:
# set up dataloaders
batch_size = 50
train_dataloader = DataLoader(train_ds, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

In [24]:
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} ")
    break

#{y.dtype}

Shape of X [N, C, H, W]: torch.Size([50, 3, 224, 224])
Shape of y: torch.Size([50]) 


### Pytorch **Neural Network**

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as dset
import torchvision.transforms as T
import torchvision.models
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data import sampler

In [26]:
from torchvision.models.densenet import densenet121

class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        densenet = torchvision.models.densenet121(pretrained=True)
        for param in densenet.parameters():
          param.requires_grad = False
        
        self.nn = nn.Sequential(
              # nn.AdaptiveAvgPool2d((224,224)),
              # nn.AvgPool2d(stride=1,kernel_size=-103)
              # cv2.resize(imgarray, (224,224), interpolation = cv2.INTER_AREA)
              *list(densenet.children())[:-1],
              nn.AdaptiveAvgPool2d((1,1))
              # nn.AvgPool2d(stride=1,kernel_size=7)
        )
        # self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        # self.avgpool= nn.AvgPool2d(x, x.size()[2:])
        # self.avgpool= nn.AvgPool2d(stride=1,kernel_size=7)
        self.out = nn.Sequential(
            nn.Linear(1024,512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512,256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256,5),
            nn.LogSoftmax(dim=1)
        )

    def forward(self,x1):
        x1 = self.nn(x1)
        # x1 = self.avgpool(x1)

        # Concatenate in dim1 (feature dimension)
        x1 = torch.flatten(x1, 1)
        scores = self.out(x1)
        return scores

In [27]:
# set up loss function and optimizer
model = MyModel()
model = model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),weight_decay=0.001,lr=0.001)
# scheduler = MultiStepLR(optimizer, milestones=[30,80], gamma=0.1)

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


  0%|          | 0.00/30.8M [00:00<?, ?B/s]

In [ ]:
print(model)

MyModel(
  (nn): Sequential(
    (0): Sequential(
      (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLayer(
          (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=T

In [28]:
# define training function

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [29]:
# define testing function
def test(dataloader, model, loss_fn,valloss):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    valloss.append(test_loss)
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    if test_loss <= min(valloss):
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(min(valloss),test_loss))
            # save checkpoint as best model
            checkpoint = checkpoint = {
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
        }
            torch.save(checkpoint,'best_model.pt')

In [ ]:
epochs = 50
valloss = []
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn,valloss)
print("Done!")

Epoch 1
-------------------------------
loss: 1.592143  [    0/12000]


#### Save preprocessor function to "preprocessor.zip" file

In [ ]:
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"") 

#### Save model to local ".onnx" file


In [ ]:
# Save pytorch model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

example_input = torch.randn(1, 3, 120, 120, requires_grad=True)

onnx_model = model_to_onnx(model, framework='pytorch',
                           model_input=example_input,
                          transfer_learning=True,
                          deep_learning=True)

with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [ ]:
import onnx
onnx_model = onnx.load("model.onnx")
onnx.checker.check_model(onnx_model)

## 4. Generate predictions from X_test data and submit model to competition

In [ ]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    # valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer

In [ ]:
model = MyModel()
model = model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),weight_decay=0.001,lr=0.001)

In [ ]:
model, optimizer = load_ckp('best_model.pt', model, optimizer)

In [ ]:
# import and preprocess X_test images in correct order...
# ...for leaderboard prediction submissions
filenumbers=[str(x) for x in range(1, 5001)]
filenames=["competition_data/testdata/test/test"+x for x in filenumbers]

#preprocess rbg images into 120,120,3 numpy ndarray
preprocessed_image_data=[]
for i in filenames:
  try:
    preprocessed_image_data.append(preprocessor(i))
  except:
    pass  

In [ ]:
X_test_submissiondata=np.vstack(preprocessed_image_data) 
tensor_X_test_submissiondata = torch.Tensor(X_test_submissiondata) 

In [ ]:
class TestDataset(torch.utils.data.Dataset):
  def __init__(self,dataset):
    self.x = dataset
  def __len__(self):
    return len(self.x)
  def __getitem__(self,idx):
    # self.x = preprocessor(filenames[i])
    # self.x = transform(self.x)
    # self.x = cv2.resize(self.x, (224,224), interpolation = cv2.INTER_AREA)
    # return self.x[idx],self.y[idx]
    return torch.squeeze(nn.functional.interpolate(self.x[idx].unsqueeze(0), (224,224)),0)

In [ ]:
test_submission = TestDataset(tensor_X_test_submissiondata)
test_dl = DataLoader(test_submission, batch_size=1, shuffle=False)
prediction_column_index = []

In [ ]:
def predict(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    with torch.no_grad():
        for X in dataloader:
            X = X.to(device)
            pred = model(X)
            prediction = pred.argmax(axis=1)
            prediction_column_index.append(prediction)

In [ ]:
predict(test_dl,model,loss_fn)

In [ ]:
# tensor_X_test_submissiondata 
# prediction_column_index=model(tensor_X_test_submissiondata).argmax(axis=1)
# prediction_column_index=outputs.argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [['forest', 'nonforest', 'snow_shadow_cloud'][i] for i in prediction_column_index]

In [ ]:
#Set credentials using modelshare.org username/password
import aimodelshare as ai
from aimodelshare.aws import set_credentials

# Note -- This is the unique rest api that powers this climate change image classification  Model Plaground
# ... Update the apiurl if submitting to a new competition

apiurl="https://srdmat3yhf.execute-api.us-east-1.amazonaws.com/prod/m"
set_credentials(apiurl=apiurl)

AI Modelshare Username:··········
AI Modelshare Password:··········
AI Model Share login credentials set successfully.


In [ ]:
#Instantiate Competition

mycompetition= ai.Competition(apiurl)

In [ ]:
#Submit Model 1: 
#-- Generate predicted y values (Model 1)
#Note: Keras predict returns the predicted column index location for classification models
# Custom metadata can be added by passing a dict to the custom_metadata argument of the submit_model() method
# This option can be used to fill in missing data points or add new columns to the leaderboard

custom_meta = {'team': 'SxB',
               'model_type': 'Sequential',
               'Deep Learning':True,
               'Optimizer':'Adam',
               'Transfer Learning':True}

mycompetition.submit_model(model_filepath = None,
                                 preprocessor_filepath=None,
                                 prediction_submission=prediction_labels,
                                 custom_metadata = custom_meta)

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 169

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1535


In [ ]:
# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 7

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1535


In [ ]:
# Get leaderboard to explore current best model architectures

# Get raw data in pandas data frame
data = mycompetition.get_leaderboard()

# Stylize leaderboard data
mycompetition.stylize_leaderboard(data)

## 5. Repeat submission process to improve place on leaderboard

*Train and submit your own models using code modeled after what you see above.*

It may also be useful to examine the architeture of models that perform particuarly well/poorly, or to compare models you've created with similar models submitted by others. Use the compare_models function in combination with the leaderboard to learn more about models that been previously submitted and potentially make decisiona about what you should do next.

In [ ]:
# Compare two or more models
data=mycompetition.compare_models([1, 5], verbose=1)
mycompetition.stylize_compare(data)

,Model_1_Layer,Model_1_Shape,Model_1_Params,Model_5_Layer,Model_5_Shape,Model_5_Params
0,Conv2D,"[None, 120, 120, 32]",416,Conv2D,"[None, 120, 120, 32]",416
1,Conv2D,"[None, 120, 120, 32]",4128,Conv2D,"[None, 120, 120, 32]",4128
2,MaxPooling2D,"[None, 60, 60, 32]",0,MaxPooling2D,"[None, 60, 60, 32]",0
3,Dropout,"[None, 60, 60, 32]",0,Dropout,"[None, 60, 60, 32]",0
4,Flatten,"[None, 115200]",0,Flatten,"[None, 115200]",0
5,Dense,"[None, 16]",1843216,Dense,"[None, 16]",1843216
6,Dropout,"[None, 16]",0,Dropout,"[None, 16]",0
7,Dense,"[None, 3]",51,Dense,"[None, 3]",51


In [ ]:
https://medium.com/analytics-vidhya/pytorch-satellite-image-classification-using-neural-networks-47f2cd1f7d82